# Install pip libraries, Login to wandb and hugging face, google login

In [1]:
!pip3 install -U huggingface_hub
!pip3 install -U accelerate
!pip3 install -U peft
!pip3 install -U transformers
!pip3 install google

import torch
import transformers
import accelerate

print(f"torch: {torch.__version__}")
print(f"transformers: {transformers.__version__}")
print(f"accelerate: {accelerate.__version__}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [2]:
!pip show accelerate
print(f"accelerate: {accelerate.__version__}")

Name: accelerate
Version: 1.3.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft
accelerate: 1.3.0


In [3]:
!pip3 install wandb
!wandb login --relogin

wand_ai = ""
api_key2 = ""

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [4]:
tk=""
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `FineTuneLLMs` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `FineTuneLLMs

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [6]:
import os
from random import randrange
from functools import partial
import torch
from transformers import (
AutoModelForCausalLM, TFAutoModel,
AutoTokenizer,
# BitsAndBytesConfig,
HfArgumentParser,
Trainer,
TrainingArguments,
DataCollatorForLanguageModeling,
EarlyStoppingCallback,
pipeline,
logging,
set_seed
)

# import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM

In [8]:
import pandas as pd

data_file_path='/content/drive/MyDrive/Thesis/training_data.csv'

df1=pd.read_csv(data_file_path)
df1.head(10)

,text,label_name,label,Unnamed: 3,Unnamed: 4
0,Receiving block blk_-1608999687919862906 src: ...,Normal,0,NaN,NaN
1,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,Normal,0,NaN,NaN
2,Receiving block blk_-1608999687919862906 src: ...,Normal,0,NaN,NaN
3,Receiving block blk_-1608999687919862906 src: ...,Normal,0,NaN,NaN
4,PacketResponder 1 for block blk_-1608999687919...,Normal,0,NaN,NaN
5,PacketResponder 2 for block blk_-1608999687919...,Normal,0,NaN,NaN
6,Received block blk_-1608999687919862906 of siz...,Normal,0,NaN,NaN
7,Received block blk_-1608999687919862906 of siz...,Normal,0,NaN,NaN
8,PacketResponder 0 for block blk_-1608999687919...,Normal,0,NaN,NaN
9,Received block blk_-1608999687919862906 of siz...,Normal,0,NaN,NaN


In [9]:
!pip install datasets
from datasets import load_dataset

# Load dataset
# dataset = load_dataset("csv", data_files = dataset_name, split = "train[:80%]")
dataset = load_dataset("csv", data_files = data_file_path, split = "train")

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Generating train split: 0 examples [00:00, ? examples/s]

Number of prompts: 104815
Column names are: ['text', 'label_name', 'label', 'Unnamed: 3', 'Unnamed: 4']


In [10]:
dataset1=dataset[0]
print(dataset[randrange(len(dataset))])

{'text': 'PacketResponder 0 for block blk_-7181549236548390449 terminating', 'label_name': 'Normal', 'label': 0, 'Unnamed: 3': None, 'Unnamed: 4': None}


In [12]:
def clean_dataset(data, seed):
  import pandas as pd
  from datasets import Dataset
  from transformers import AutoTokenizer
  from functools import partial

  data = data.remove_columns(["Unnamed: 3", "Unnamed: 4"])

  # Convert DataFrame to Hugging Face Dataset
  hf_dataset = data

  # Step 3: Tokenize the dataset
  def tokenize_function(batch, tokenizer, max_length):
      return tokenizer(
          batch["text"],
          max_length=max_length,
          truncation=True,
      )

  # Load tokenizer
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  max_length = get_max_length(model)

  # Tokenize the dataset
  preprocessing_function = partial(tokenize_function, tokenizer=tokenizer, max_length=max_length)
  tokenized_dataset = hf_dataset.map(preprocessing_function, batched=True, remove_columns = ["text", "label_name"],)
  tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

  # Step 4: Shuffle and finalize dataset
  # Shuffle dataset for better training
  tokenized_dataset = tokenized_dataset.shuffle(seed=seed)

  return tokenized_dataset

seed = 42
# Final dataset ready for fine-tuning
preprocessed_dataset = clean_dataset(dataset,seed)
train_test = preprocessed_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test["train"]
valid_dataset = train_test["test"]


print(preprocessed_dataset)

Found max lenth: 514


Map:   0%|          | 0/104815 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 104815
})


# Load Model


In [29]:
def load_model(model_name):
  from transformers import RobertaTokenizerFast
  from transformers import AutoModelForSequenceClassification

  #Loads model and model tokenizer

  #:param model_name: Hugging Face model name
  #:param bnb_config: Bitsandbytes configuration"""

  # Get number of GPU device and set maximum memory
  n_gpus = torch.cuda.device_count()
  max_memory = f'{40960}MB'

  device = "cuda" if torch.cuda.is_available() else "cpu"
  print(device)


  # Load model
  model = AutoModelForSequenceClassification.from_pretrained(
  model_name,
  torch_dtype=torch.float32,
  device_map = 'auto', # dispatch the model efficiently on the available resources
  max_memory = {i: max_memory for i in range(n_gpus)},
  )
  model.to("cuda")

  # model = TFAutoModel.from_pretrained(model_name)

  # Load model tokenizer with the user authentication token
  tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

  # Set padding token as EOS token
  tokenizer.pad_token = tokenizer.eos_token
  return model, tokenizer

model_name='FacebookAI/roberta-base'
# model_name='meta-llama/Llama-2-7b-hf'

model, tokenizer = load_model(model_name)


cuda


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


# Utility Methods

In [30]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [31]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r=r,
        lora_alpha=lora_alpha,
        target_modules=target_modules,
        lora_dropout=lora_dropout,
        bias=bias,
        task_type=task_type,
        modules_to_save=None

    )
    return config

In [32]:
def find_all_linear_names(model):
  """
  Find modules to apply LoRA to.

  :param model: PEFT model
  """
  lora_module_names = set()
  for name, module in model.named_modules():
      if isinstance(module, torch.nn.Linear):
          names = name.split('.')
          lora_module_names.add(names[0] if len(names) == 1 else names[-1])
          module_name = names[0] if len(names) == 1 else names[-1]
          # Exclude classifier layers from LoRA
          if "classifier" not in name:
              lora_module_names.add(module_name)

  exclude_layers = {'classifier', 'dropout', 'default', 'out_proj', 'dense'}
  lora_module_names = lora_module_names - exclude_layers
  # if 'classifier' in lora_module_names:
      # lora_module_names.remove('classifier')  # Exclude final classification head
  print(f"LoRA module names: {list(lora_module_names)}")
  return list(lora_module_names)

In [33]:
def print_trainable_parameters(model, use_4bit = False):
  """
  Prints the number of trainable parameters in the model.

  :param model: PEFT model
  """

  trainable_params = 0
  all_param = 0

  for _, param in model.named_parameters():
      num_params = param.numel()
      all_param += num_params
      if param.requires_grad:
          trainable_params += num_params

  print(
      f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param:.2f}%"
  )

# Fine tuning the model


In [49]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 128

# Dropout probability for LoRA layers
lora_dropout = 0.05

# Bias
bias = "none"

# Task type
task_type = "SEQ_CLS"

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 16

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 2

# Initial learning rate (AdamW optimizer)
learning_rate = 5e-5

# Optimizer to use
optim = "adamw_torch"

# Number of training steps (overrides num_train_epochs)
max_steps = 10000

# Linear warmup steps from 0 to learning_rate
warmup_steps = 100

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = True

# Log every X updates steps
logging_steps = 100

# Save checkpoints every X steps
save_steps = 1000  # Saves progress every 100 steps

# Evaluation strategy (Evaluate every few steps)
evaluation_strategy = "steps"
eval_steps = 1000  # Evaluate every 100 steps


In [50]:
def fine_tune(model,
        tokenizer,
        dataset,
        valid_dataset,
        lora_r,
        lora_alpha,
        lora_dropout,
        bias,
        task_type,
        per_device_train_batch_size,
        gradient_accumulation_steps,
        warmup_steps,
        max_steps,
        learning_rate,
        fp16, bf16,
        logging_steps,
        output_dir,
        optim):
  """
  Prepares and fine-tune the pre-trained model.

  :param model: Pre-trained Hugging Face model
  :param tokenizer: Model tokenizer
  :param dataset: Preprocessed training dataset
  """

  from transformers import DataCollatorWithPadding

  # Enable gradient checkpointing to reduce memory usage during fine-tuning
  model.gradient_checkpointing_enable()

  # Prepare the model for training
  # model = prepare_model_for_kbit_training(model)

  # Get LoRA module names
  target_modules = find_all_linear_names(model)

  # Create PEFT configuration for these modules and wrap the model to PEFT
  peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
  model = get_peft_model(model, peft_config)

  # Print information about the percentage of trainable parameters
  print_trainable_parameters(model)

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  # Training parameters
  trainer = Trainer(
      model = model,
      train_dataset = dataset,
      eval_dataset=valid_dataset,
      args = TrainingArguments(
          per_device_train_batch_size = per_device_train_batch_size,
          per_device_eval_batch_size=32,
          gradient_accumulation_steps = gradient_accumulation_steps,
          warmup_steps = warmup_steps,
          max_steps = max_steps,
          learning_rate = learning_rate,
          fp16 = fp16,
          # bf16=bf16,
          logging_steps = logging_steps,
          output_dir = output_dir,
          optim = optim,
          evaluation_strategy="steps",
          eval_steps=eval_steps,
          save_steps=save_steps,
          save_total_limit=2,  # Keep only last 2 checkpoints
          load_best_model_at_end=True,
      ),
      data_collator = data_collator
  )

  model.config.use_cache = False

  do_train = True

  # Launch training and log metrics
  print("Training...")

  if do_train:
      train_result = trainer.train()
      print(f"✅ Training complete...!! \n {train_result}")
      metrics = train_result.metrics
      trainer.log_metrics("train", metrics)
      trainer.save_metrics("train", metrics)
      trainer.save_state()
      print(metrics)

  # Save model
  print("Saving last checkpoint of the model...")
  os.makedirs(output_dir, exist_ok = True)

  # Save using both PEFT and base model
  trainer.model.save_pretrained(output_dir, safe_serialization=True)
  trainer.model.base_model.save_pretrained(os.path.join(output_dir, "base_model"))
  # model.save_model(output_dir)

  # Free memory for merging weights
  del model
  del trainer
  torch.cuda.empty_cache()

In [18]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

CUDA Available: True
GPU Device Name: Tesla T4


# Training

In [ ]:
fine_tune(model,
          tokenizer,
          train_dataset,
          valid_dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,bf16,
          logging_steps,
          output_dir,
          optim
          )

LoRA module names: ['base_layer']
All Parameters: 146,472,964 || Trainable Parameters: 21,825,794 || Trainable Parameters %: 14.90%
Training...


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.067000,0.064178
2000,0.029800,0.034259


Step,Training Loss,Validation Loss
1000,0.067000,0.064178
2000,0.029800,0.034259


In [ ]:
type(model)

# Save and Download

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("results", 'zip', "results")
files.download("results.zip")

# Testing

In [ ]:
text_to_generate = "Receiving block blk_-3544583377289625738 src: /10.250.19.102:39325 dest: /10.250.19.102:50010"

# Tokenize the input text
input_ids = tokenizer.encode(text_to_generate, return_tensors="pt")
input_ids = input_ids.to('cuda')
# Generate output from the model
output = model.generate(input_ids,max_new_tokens=40)

# Decode the generated output
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)

In [ ]:
from transformers.agents import LocalAgent

agent = LocalAgent(model,tokenizer)
agent.run("Categorize the following text into one of the 2 categories:\n\nNormal\nAnomaly", text=text_to_generate)


In [ ]:
from transformers import pipeline

text_to_generate = "Receiving block blk_-3544583377289625738 src: /10.250.19.102:39325 dest: /10.250.19.102:50010"

# Load text classification pipeline
classifier = pipeline("text-classification", model=model_name)

# Predict category
result = classifier(text_to_generate)
print(result)

In [ ]:
result = classifier("10.250.11.100:50010 Served block blk_-3544583377289625738 to /10.250.19.102")
print(result)

# BertViz

In [ ]:
! pip3 install bertviz

In [ ]:
from bertviz import model_view, head_view

text_to_generate = "10.250.11.100:50010 Served block blk_-3544583377289625738 to /10.250.19.102"

# Tokenize the input text
input_ids = tokenizer.encode(text_to_generate, return_tensors="pt").to('cuda')
# Generate output from the model
output = model(input_ids, output_attentions=True)

attentions = output.attentions
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

# Debug: Print attention shapes
for i, layer_attention in enumerate(attentions):
    print(f"Layer {i}: {layer_attention.shape}")  # Should be (batch_size, num_heads, seq_len, seq_len)



In [ ]:
type(tokens)

In [ ]:
head_view(attentions, tokens)

In [ ]:
model_view(attentions, tokens)